# Analisis De Textos

1. De que tratan las empresas que se estan consitituyendo? NLP en descripcion de actividades.
2. Hay algun patron geografico? (x ej con https://kepler.gl/)
3. Hay cambios a lo largo del tiempo? Algún peak interesante?
4. Qué características de empresa tienden a pedir cuenta corriente y cuáles no?
5. Cuánto tiempo pasa entre que una empresa se constituye y solicitan una cuenta corriente?
6. Podemos ver alguna red de RUTs? Gente que esta relacionada en varias empresas (quizas con https://www.graphext.com/)


In [1]:
#!pip install awswrangler
#!pip install --upgrade seaborn
#!pip install wordcloud
#!pip install --upgrade pandas
#!pip install unidecode
#!pip install gensim

In [5]:
import boto3

session = boto3.session.Session(profile_name='legalbot')
s3 = session.resource('s3')

In [6]:
import pandas as pd
import awswrangler as wr
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme()
sns.set_context('poster')

from matplotlib import rcParams
rcParams['figure.figsize'] = 15,8

In [7]:
bucket='legalres-storage-study-json'
prefix = 'processed'
data_location = 's3://{}/{}/'.format(bucket, prefix)

In [8]:
df = wr.s3.read_parquet(path=data_location,
                        boto3_session=session)

In [9]:
df.shape

(798060, 7)

In [10]:
df.columns

Index(['rut', 'cve', 'fecha', 'lugar', 'tipo_sociedad', 'objeto', 'firmas'], dtype='object')

In [11]:
df.head()

,rut,cve,fecha,lugar,tipo_sociedad,objeto,firmas
0,76304233-2,AC1DrSkE1pdl,2013-10-08,"En TEMUCO, Región de LA ARAUCANIA, Chile, a 08...",EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA,La Empresa tendrá por objeto desarrollar las s...,"[{'anotacion': None, 'firmado': 'Firmado elect..."
1,76304089-5,AC1Dv0J0V4Zr,2013-10-07,"En CONCEPCION, Región del BIOBIO, Chile, a 07 ...",SOCIEDAD DE RESPONSABILIDAD LIMITADA,La Sociedad tendrá por objeto desarrollar las ...,"[{'anotacion': None, 'firmado': 'Firmado elect..."
2,76313238-2,AC1G6RzRn16K,2013-10-24,"En MOLINA, Región del MAULE, Chile, a 24 de oc...",SOCIEDAD DE RESPONSABILIDAD LIMITADA,La Sociedad tendrá por objeto desarrollar las ...,"[{'anotacion': None, 'firmado': 'Firmado elect..."
3,76313389-3,AC1G8rKdG7Xu,2013-10-24,"En COPIAPO, Región de ATACAMA, Chile, a 24 de ...",SOCIEDAD DE RESPONSABILIDAD LIMITADA,La Sociedad tendrá por objeto desarrollar las ...,"[{'anotacion': None, 'firmado': 'Firmado elect..."
4,76313805-4,AC1H6Pt61S4o,2013-10-29,"En TALCA, Región del MAULE, Chile, a 29 de oct...",EMPRESA INDIVIDUAL DE RESPONSABILIDAD LIMITADA,La Empresa tendrá por objeto desarrollar las s...,"[{'anotacion': None, 'firmado': 'Firmado elect..."


## Analisis de Texto 

In [4]:
from bertopic import BERTopic

In [2]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
#model = SentenceTransformer('dccuchile/bert-base-spanish-wwm-uncased')




In [22]:
topic_model = BERTopic(embedding_model=model, language='spanish',
                       n_gram_range=(1, 3), verbose=True)

In [23]:
topics, probs = topic_model.fit_transform(df['objeto'].sample(30000).to_list())

2022-04-18 23:15:43,096 - BERTopic - Transformed documents to Embeddings


OMP: Info #270: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
2022-04-18 23:16:19,091 - BERTopic - Reduced dimensionality with UMAP


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

2022-04-18 23:16:23,276 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [27]:
topic_model.visualize_topics(top_n_topics=30)